In [2]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt

from zipfile import ZipFile
from urllib.request import urlretrieve

%matplotlib inline

In [3]:
def download_and_unzip(url, save_path):
    print(f"Downloading and extracting assests....", end="")

    urlretrieve(url, save_path)

    try:
        with ZipFile(save_path) as z:
            z.extractall(os.path.split(save_path)[0])

        print("Done")

    except Exception as e:
        print("\nInvalid file.", e)
URL = r"https://www.dropbox.com/s/qa1hsyxt66pvj02/opencv_bootcamp_assets_NB10.zip?dl=1"

asset_zip_path = os.path.join(os.getcwd(), "opencv_bootcamp_assets_NB10.zip")

if not os.path.exists(asset_zip_path):
    download_and_unzip(URL, asset_zip_path)

Basic Idea:
1. Dynamic range of images is limited to 8-bits (0 - 255) per channel
2. very bright pixels saturate to 255
3. very dark pixels clip to 0

Step 1: Capture Multiple Exposures

In [4]:
def readImagesAndTimes():
    filenames = ["img_0.033.jpg", "img_0.25.jpg", "img_2.5.jpg", "img_15.jpg"]
    times = np.array([1 / 30.0, 0.25, 2.5, 15.0], dtype=np.float32)
    images = []
    for filename in filenames:
        im = cv2.imread(filename)
        images.append(im)
    return images, times

In [5]:
images, times = readImagesAndTimes()

alignMTB = cv2.createAlignMTB()
alignMTB.process(images, images)

[ WARN:0@0.411] global loadsave.cpp:241 findDecoder imread_('img_0.033.jpg'): can't open/read file: check file path/integrity
[ WARN:0@0.411] global loadsave.cpp:241 findDecoder imread_('img_0.25.jpg'): can't open/read file: check file path/integrity
[ WARN:0@0.411] global loadsave.cpp:241 findDecoder imread_('img_2.5.jpg'): can't open/read file: check file path/integrity
[ WARN:0@0.411] global loadsave.cpp:241 findDecoder imread_('img_15.jpg'): can't open/read file: check file path/integrity


error: OpenCV(4.10.0) /Users/xperience/GHA-Actions-OpenCV/_work/opencv-python/opencv-python/opencv/modules/imgproc/src/color.cpp:196: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'


images, times = readImagesAndTimes() reads the images and then the exposure times too.

The second half aligns the images.

Step 3: Estimate Camera Response Function

In [ ]:
calibrateDebevec = cv2.createCalibrateDebevec()
responseDebevec = calibrateDebevec.process(images, times)

x = np.arange(256, dtype=np.uint8)
y = np.squeez(responseDebevec)

ax = plt.figure(figsize=(30, 10))
plt.title("Debevec Inverse Camera Response Function", fontsize=24)
plt.xlabel("Measured Pixel Value", fontsize=22)
plt.ylabel("Calibrated Intensity", fontsize=22)
plt.xlim([0, 260])
plt.grid()
plt.plot(x, y[:, 0], "b", x, y[:, 1], "g", x, y[:, 2], "r")
plt.show()

NameError: name 'images' is not defined

Step 4: Merge Exposure into an HDR Image:

In [6]:
mergeDebevec = cv2.createMergeDebevec()
hdrDebevec = mergeDebevec.process(images, times, responseDebevec)

NameError: name 'responseDebevec' is not defined

Merges images into a HDR Linear image

Step 5: Tonemapping

In [ ]:
tonemapDrago = cv2.createTonemapDrago(1.0, 0.7)
ldrDrago = tonemapDrago.process(hdrDebevec)
ldrDrago = 3 * ldrDrago
cv2.imwrite("ldr-Drago.jpg", 255*ldrDrago)
plt.figure(figsize=(20, 10));plt.imshow(np.clip(ldrDrago, 0, 1)[:,:,::-1]);plt.axis("off");
#Using Drago Method to obtain 24-bit color image

In [ ]:
print("Tonemaping using Reinhard's method ... ")
tonemapReinhard = cv2.createTonemapReinhard(1.5, 0, 0, 0)
ldrReinhard = tonemapReinhard.process(hdrDebevec)
cv2.imwrite("ldr-Reinhard.jpg", ldrReinhard * 255)
plt.figure(figsize=(20, 10));plt.imshow(np.clip(ldrReinhard, 0, 1)[:,:,::-1]);plt.axis("off")
#Using Reinhard's method to obtain 24-bit color image

In [ ]:
print("Tonemaping using Mantiuk's method ... ")
tonemapMantiuk = cv2.createTonemapMantiuk(2.2, 0.85, 1.2)
ldrMantiuk = tonemapMantiuk.process(hdrDebevec)
ldrMantiuk = 3 * ldrMantiuk
cv2.imwrite("ldr-Mantiuk.jpg", ldrMantiuk * 255)
plt.figure(figsize=(20, 10));plt.imshow(np.clip(ldrMantiuk, 0, 1)[:,:,::-1]);plt.axis("off")
#Using Mantiuk's method to obtain 24-bit color image

The purpose of cv2.createAlignMTB is to create an object for aligning images based on brightness differences.

The purpose of the tone mapping process is to reduce the dynamic range of HDR image for display on a low dynamic range monitor.

cv2.createMergeDebevec handles over and under exposed pixels by using an adaptive weighting function

The potential issue when using cv2.createMergeDebevec is the motion blur in the imput images.